# Chattbottar - Kapitel 10 

I detta kodexempel ska vi få en grundläggande förståelse för hur chattbottar fungerar. Let's go! 

In [1]:
# pip install  google-genai
# pip install pypdf

In [3]:
import numpy as np
from google import genai
from pypdf import PdfReader

# Använda en chattbott genom API
Vi kommer nu demonstrera hur vi kan ha en egen chattbott genom att använda en API-nyckel från Google i detta fall. Andra alternativ är t.ex. ChatGPT/OpenAI. 
Om du i verkligheten ska arbeta med chattbottar kommer du mest sannolikt att använda färdiga API. 

För att skapa en API-nyckel behöver du gå in på följande hemsids: https://aistudio.google.com/ och sen gå till "Create API key". Notera, du behöver registrera ditt bankkort för att få en gratis provperiod. Annars funkar det inte. Som vanlig praxis inom systemutveckling så delar du inte din privata API-nyckel eftersom då kan andra använda den vilket kostar pengar. Man kan använda "api_key = api_key=os.getenv("API_KEY")" för detta ändamål. I koden nedan **skriver jag explicit ut min API-nyckel som du dock inte kommer kuna använda (du behöver skapa en egen)**. 

In [ ]:
# import os
# api_key = api_key=os.getenv("API_KEY")

api_key = 'HEMLIGT'
client = genai.Client(api_key=api_key)

response = client.models.generate_content(
    model="gemini-3-flash-preview", 
    contents="Skriv i 5 meningar vad som kännetecknar en skicklig programmerare."
)

print(response.text)

Här är fem kännetecken för en skicklig programmerare:

1. En skicklig programmerare har en stark logisk förmåga och kan effektivt bryta ner komplexa problem i mindre, hanterbara delar.
2. De skriver läsbar och välstrukturerad kod som inte bara löser uppgiften, utan även är enkel för andra att förstå och underhålla.
3. Genom en ständig nyfikenhet och vilja att lära sig nya tekniker håller de sig uppdaterade i en bransch som är i ständig förändring.
4. De besitter ett stort tålamod vid felsökning och strävar efter att förstå grundorsaken till problem snarare än att bara applicera tillfälliga lösningar.
5. Slutligen är god kommunikationsförmåga och ödmjukhet inför andras idéer avgörande, eftersom framgångsrik mjukvaruutveckling oftast bygger på ett bra lagarbete.


# Skapa en mycket enkel applikation
Vi kan också skapa en enkel applikation. Men lite kreativitet så inser man att det finns många möjligheter att bygga/utveckla sådant man är intresserad av.

In [6]:
print("*** Gemini chat ***")
print("Type <q> to exit chat.")

while True:
    prompt = input("User: ")
    if prompt == "q":
        break
    else:
        response = client.models.generate_content(
            model="gemini-3-flash-preview",
            contents=prompt
        )
        print("Gemini:", response.text)

*** Gemini chat ***
Type <q> to exit chat.


Gemini: None
Gemini: Hej! Vad kan jag hjälpa dig med idag?


# Retrieval Augmented Generation (RAG)

## Läsa in PDF-fil

Vi börjar med att läsa in en PDF-fil som chattbotten kommer ge svar utifrån. 

In [7]:
reader = PdfReader("chattbot.pdf")

text = ""
for page in reader.pages:
    text += page.extract_text()

In [8]:
print(len(text))

23416


In [7]:
print(text[17:550])

Kapitel 1
Chattbot
1.1 Chattbottar
I detta kapitel kommer vi lära oss mer om hur chattbottar såsom ChatGPT fungerar och
hur man ställer effektiva frågor till dem, vilket kallas prompt engineering . Vi kommer
därefter lära oss hur vi bygger en lokal chattbot som vi kan använda på vår egna dator.
Kapitlet avslutas med att gå igenom RAG vilket låter oss anpassa chattbottens svar
utifrån en given kontext, exempelvis utifrån egna dokument.
1.1.1 ChatGPT och prompt engineering
Chattbottar, som till exempel ChatGPT, används av många m


## Chunking

### Fixed length-chunking

In [9]:
chunks = []
n = 1000
overlap = 200
for i in range(0, len(text), n - overlap):
    chunks.append(text[i:i + n])

print(f"Antal chunks: {len(chunks)}.")

Antal chunks: 30.


In [9]:
print(chunks[0])

Chattbot med RAG2Kapitel 1
Chattbot
1.1 Chattbottar
I detta kapitel kommer vi lära oss mer om hur chattbottar såsom ChatGPT fungerar och
hur man ställer effektiva frågor till dem, vilket kallas prompt engineering . Vi kommer
därefter lära oss hur vi bygger en lokal chattbot som vi kan använda på vår egna dator.
Kapitlet avslutas med att gå igenom RAG vilket låter oss anpassa chattbottens svar
utifrån en given kontext, exempelvis utifrån egna dokument.
1.1.1 ChatGPT och prompt engineering
Chattbottar, som till exempel ChatGPT, används av många människor för en rad olika
syften. Alltifrån att skapa kreativt innehåll, få förslag på förbättringar av skriven text
eller programmeringskod till att lösa mer komplexa problem.
I korthet har chattbottar tränats på enorma datamängder och har från detta lärt sig vad
som är rimliga sekvenser av ord. Exempelvis vet vi att om någon säger “Hej, hur mår
___” så brukar det sista ordet vara “du” för att meningen ska bli “Hej, hur mår du” .
När vi ställer 

In [10]:
print(text[26:584])


Chattbot
1.1 Chattbottar
I detta kapitel kommer vi lära oss mer om hur chattbottar såsom ChatGPT fungerar och
hur man ställer effektiva frågor till dem, vilket kallas prompt engineering . Vi kommer
därefter lära oss hur vi bygger en lokal chattbot som vi kan använda på vår egna dator.
Kapitlet avslutas med att gå igenom RAG vilket låter oss anpassa chattbottens svar
utifrån en given kontext, exempelvis utifrån egna dokument.
1.1.1 ChatGPT och prompt engineering
Chattbottar, som till exempel ChatGPT, används av många människor för en rad olika
syften. 


## Embeddings
Se t.ex. s.321 i kursboken "Lär dig AI från grunden - Tillämpad maskininlärning med Python" för vad Embeddings innebär. I korthet, ord representeras med vektorer/siffror. 

In [10]:
from google.genai import types

def create_embeddings(text, model="text-embedding-004", task_type="SEMANTIC_SIMILARITY"): 
    return client.models.embed_content(model=model, contents=text, config=types.EmbedContentConfig(task_type=task_type))

In [11]:
embeddings = create_embeddings(chunks)
len(embeddings.embeddings)

30

In [12]:
len(embeddings.embeddings)

embeddings.embeddings[0].values[0:10]

[-0.057309315,
 -0.010547505,
 -0.08322796,
 0.024299115,
 0.055630915,
 0.048652556,
 0.049652215,
 -0.027345037,
 0.044459436,
 -0.042770572]

## Semantisk sökning

In [1]:
def cosine_similarity(vec1, vec2):
    return (np.dot(vec1, vec2) / (np.linalg.norm(vec1)*np.linalg.norm(vec2)))

In [14]:
def semantic_search(query, chunks, embeddings, k=5):
    query_embedding = create_embeddings(query).embeddings[0].values 
    similarity_scores = []
    
    for i, chunk_embedding in enumerate(embeddings.embeddings):
        similarity_score = cosine_similarity(query_embedding, chunk_embedding.values)
        similarity_scores.append((i, similarity_score))

    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    top_indices = [index for index, _ in similarity_scores[:k]]
    
    return [chunks[index] for index in top_indices]

In [16]:
fråga = "Vad kan RAG användas till?"
svar = semantic_search(fråga, chunks=chunks, embeddings=embeddings, k=1)
print(svar)

['i ger\nmodellen en kontext, ofta ett eller flera dokument eller liknande, att förhålla sig till.\nI en prompt säger vi åt modellen att enbart svara utifrån den givna kontexten. Om\nmodellen inte hittar svaret i kontexten ska den säga det istället för att försöka hitta\nsvaren någon annanstans eller gissa.\n7En RAG-modell innehåller alltså två delar.\nDen första delen är en retriever, som söker efter relevanta stycken i en större text.\nDessa stycken skickas sedan vidare som kontext till den andra delen, en generator som\ngenererar svaren utifrån den givna kontexten.\nFör att ge modellen en kontext behöver vi läsa in data (exempelvis ett eller flera PDF-\ndokument) och bearbeta den så att vi kan göra en semantisk sökning i datan, det vill\nsäga leta upp de stycken i kontexten som verkar ha mest med själva frågan att göra.\nDessa stycken skickar vi sedan med till språkmodellen när vi ställer vår fråga.\nDet finns ett antal ramverk för att implementera RAG, bland annat LangChain. Vi\nko

## Generera bra svar med RAG

In [15]:
system_prompt = """Jag kommer ställa dig en fråga, och jag vill att du svarar
baserat bara på kontexten jag skickar med, och ingen annan information.
Om det inte finns nog med information i kontexten för att svara på frågan,
säg "Det vet jag inte". Försök inte att gissa.
Formulera dig enkelt och dela upp svaret i fina stycken. """

In [16]:
def generate_user_prompt(query):
    context = "\n".join(semantic_search(query, chunks, embeddings))
    user_prompt = f"Frågan är {query}. Här är kontexten: {context}."
    return user_prompt

In [17]:
def generate_response(system_prompt, user_message, model="gemini-3-flash-preview"):
    response = client.models.generate_content(
        model=model,
        config=genai.types.GenerateContentConfig(
            system_instruction=system_prompt),
            contents=generate_user_prompt(user_message)
        )
    return response

In [18]:
print(generate_response(system_prompt, "Vad är RAG?").text)

En RAG-modell är ett system där man ger en språkmodell en specifik kontext, till exempel ett eller flera dokument, som den ska förhålla sig till när den svarar på frågor. Syftet är att modellen ska svara enbart utifrån den givna informationen istället för att gissa eller hämta svar från andra källor.

Modellen består av två huvuddelar: en **retriever** och en **generator**. Retrieverns uppgift är att söka igenom en större mängd data och leta upp de stycken som är mest relevanta för frågan som ställts. Detta görs ofta genom en så kallad semantisk sökning.

De relevanta styckena skickas sedan vidare till generatorn. Generatorn skapar därefter ett svar baserat på den utvalda kontexten. RAG-tekniken kan användas med många olika typer av data, exempelvis text från PDF-dokument.


In [19]:
fråga = "Vad är meningen med livet?"
svar = generate_response(system_prompt, fråga).text
print(svar)

Det vet jag inte.


In [20]:
fråga = "Vad är prompt engineering?"
svar = generate_response(system_prompt, fråga).text
print(svar)

Prompt engineering handlar om hur man ställer effektiva frågor till chattbottar, såsom ChatGPT. Syftet med detta är att få "bättre" och mer användbara svar från språkmodellen.

Rent praktiskt går det ut på att man som användare är så specifik, beskrivande och detaljerad som möjligt. Man kan till exempel ge information om önskad kontext, stil, format och längd på svaret för att styra chattbotten rätt.


# Evaluering

In [21]:
validation_data = [
{"question": "Vilka delar utgör en RAG-modell?",
"ideal_answer": """En RAG-modell innehåller två delar: 
en retriever som söker efter relevanta stycken i en text, 
och en generator som genererar svar utifrån den givna kontexten."""}
]

print(validation_data[0]["question"])
print()
print(validation_data[0]["ideal_answer"])

Vilka delar utgör en RAG-modell?

En RAG-modell innehåller två delar: 
en retriever som söker efter relevanta stycken i en text, 
och en generator som genererar svar utifrån den givna kontexten.


In [22]:
evaluation_system_prompt = """Du är ett intelligent utvärderingssystem vars uppgift är att utvärdera en AI-assistents svar. 
Om svaret är väldigt nära det önskade svaret, sätt poängen 1. Om svaret är felaktigt eller inte bra nog, sätt poängen 0.
Om svaret är delvis i linje med det önskade svaret, sätt poängen 0.5. Motivera kort varför du sätter den poäng du gör.
"""

In [23]:
query = validation_data[0]["question"]

response = generate_response(system_prompt, query)

evaluation_prompt = f"""Fråga: {query}
AI-assistentens svar: {response.text}
Önskat svar: {validation_data[0]['ideal_answer']}"""

# Note, we have created a "evaluation_system_prompt" and a "evaluation_prompt" that we use in our function "generate_response" that we created before. 
evaluation_response = generate_response(evaluation_system_prompt, evaluation_prompt)
print(evaluation_response.text)

Poäng: 1

Motivering: AI-assistentens svar är korrekt och innehåller precis de två delar som efterfrågas (retriever och generator) med förklaringar som stämmer väl överens med det önskade svaret.


In [24]:
# Try change to "ideal_answer": "Java" and notice that if the user defines the wrong "ideal answer", then it gets weird. 
# In a wider context, how do you define ideal answers to questions that have no exact answer and who does this? 

validation_data_2 = [
{"question": "Vilket programmeringsspråk är kapitlet skrivet i?",
"ideal_answer": "Python"}
]
print(validation_data_2[0]["question"])
print(validation_data_2[0]["ideal_answer"])

Vilket programmeringsspråk är kapitlet skrivet i?
Python


In [25]:
query = validation_data_2[0]["question"]

response = generate_response(system_prompt, query)
print(response.text)

Programmeringsspråket som kapitlet använder är Python.

Detta framgår då texten nämner att de kommer att använda sig av vanliga Python-bibliotek som "pypdf" för att läsa in text och "numpy" för beräkningar. 

Även kodexemplen i texten, som till exempel `from pypdf import PdfRead`, är skrivna i Python.


In [27]:
query5 = "Vad är hamburgare för något?"

response = generate_response(system_prompt,query5)
print(response.text)

Det vet jag inte.


In [28]:
evaluation_prompt = f"""Fråga: {query}
AI-assistentens svar: {response.text}
Önskat svar: {validation_data_2[0]['ideal_answer']}"""

evaluation_response = generate_response(evaluation_system_prompt, evaluation_prompt)
print(evaluation_response.text)

Poäng: 1

Motivering: Svaret är korrekt och välmotiverat utifrån den givna kontexten.


# Fördjupning
Den som är intresserad av att arbeta med chattbottar för t.ex. del 2 av kunskapskontrollen kan fördjupa sig inom LangChain, se t.ex. här: 
https://academy.langchain.com/collections/foundation

Vill man få en snabb överblick, se t.ex. "Quickstart LangChain Essentials - Python" här: https://academy.langchain.com/collections/quickstart

Vill man få en överblick över LangChain, LangGraph och LangSmith, se här: https://www.youtube.com/watch?v=vJOGC8QJZJQ